# Who is heading for Diabetes?

This is the predictive part of the 2017 Melbourne Datathon.

The task is to predict the probability that a patient will be dispensed a drug related to Diabetes post 2015. This is quite important research as it will be an early warning system for doctors so intervention can potentially be made before it is too late.

Use the patients that we have provided all the records for to build your model, then see how it performs on these unseen people.

For patient ID'S 279,201 to 558,352 you need to submit a file with 2 columns, the Patient_ID and the probability in the range [0-1]. The file will have 279,153 rows including the header row. An example submission file is provided for download.

In [33]:
import tqdm

import pandas as pd
import numpy as np
import pickle
import sqlite3
import xgboost

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report
from sklearn.svm import SVC
from sklearn.preprocessing import normalize, StandardScaler

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier

## Data retrieval functions

In [2]:
conn = sqlite3.connect("../data/datasets.db")

In [65]:
def retrieve_patient_data(connection, patient_id):
    """
    Return the patient data.
    """
    SQL = """
SELECT *
FROM transactions a
LEFT OUTER JOIN ChronicIllness_LookUp b 
    ON a.Drug_ID = b.MasterProductID 
LEFT OUTER JOIN patients c
    ON a.Patient_ID = c.Patient_ID
LEFT OUTER JOIN classification d
    ON a.Patient_ID = d.Patient_ID
LEFT OUTER JOIN social e
    ON c.postcode = e.postcode
WHERE a.Patient_ID = {}
AND a.prescription_week < '2016-01-01'
ORDER BY prescription_week
    """.format(patient_id)

    return pd.read_sql_query(SQL, connection)


def retrieve_chronic_illness(connection):
    SQL = """
SELECT DISTINCT ChronicIllness FROM ChronicIllness_LookUp 
    """.format(patient_id)
    return pd.read_sql_query(SQL, connection)
    
    

## Feature extraction functions

In [80]:
gender_map = {'F': 1, 'M': 0, 'U': 0.5}

def gender(patient_frame):
    return gender_map[patient_frame.gender[0]]

def age(patient_frame):
    patient_age = 2016 - patient_frame.year_of_birth[0]
    if patient_age > 100: 
        return 0.5 
    else: 
        return patient_age / 100.

def socio_score(patient_frame):
    score = patient_frame.disadvantage_score[0]
    if isinstance(score, str):
        return 1.
    if score is None:
        return 1.
    return float(score) / 1000.

def diabetes(patient_frame):
    return float(patient_frame.ChronicIllness.str.contains('Diabetes').any())

def mean_script_time(patient_frame):
    script_week_diff = pd.to_datetime(patient_frame.Prescription_Week).diff()
    return script_week_diff[script_week_diff > pd.Timedelta(0)].mean() / pd.Timedelta(days=365*6)

def system_codes(patient_frame):
    codes = {
         'A': 0, # Unknown
         'C': 0, # Unknown   
         'Z': 0, # Unknown
         'N': 0, # NHS Script
         'P': 0, # Private Script
         'B': 0, # Doctors Bag Script
         'T': 0, # Schedule Three Recordable Script
         'R': 0, # Repatriation Script
         'D': 0, # Dental Script
         'E': 0, # Optometrist Script
         'U': 0, # Nurse Practitioner Script
         'F': 0  # Midwife Script
        }

    codes.update(patient_frame.SourceSystem_Code.value_counts(normalize=True).to_dict())
    return codes

def chronic_illness(patient_frame):
    chronic_treatments = {
     'Anti-Coagulant': 0,
     'Chronic Obstructive Pulmonary Disease (COPD)': 0,
     'ChronicIllness': 0,
     'Depression': 0,
     'Diabetes': 0,
     'Epilepsy': 0,
     'Heart Failure': 0,
     'Hypertension': 0,
     'Immunology': 0,
     'Lipids': 0,
     'Osteoporosis': 0,
     'Urology': 0}
    chronic_treatments.update(patient_frame.ChronicIllness.value_counts(normalize=True).to_dict())
    return chronic_treatments

## Compute some basic features of the data 

In [82]:
def feature_extract(patient_frame):
    """
    Form a feature dictionary
    """
    
    feature_dict = {
        'gender': gender(patient_frame), 
        'age': age(patient_frame), 
        'socio_score': socio_score(patient_frame),
        'mean_script_time': mean_script_time(patient_frame),
        'target': patient_frame.Target[0]}
    
    feature_dict.update(system_codes(patient_frame))
    feature_dict.update(chronic_illness(patient_frame)) 
    
    return feature_dict
    

## Extract features of the data

Perform a random sample of patients.

In [85]:
n = 100000
patient_ids = np.random.randint(0, 279201, n)
patient_data = []
for patient_id in tqdm.tqdm_notebook(patient_ids):
    patient_data.append(retrieve_patient_data(conn, patient_id))

Extract features into a feature dataframe.

In [86]:
features = []
for patient in tqdm.tqdm_notebook(patient_data): 
    if len(patient):
        features.append(feature_extract(patient))
feature_frame = pd.DataFrame(features)

Store the features and ?patient data? to disk.

In [8]:
feature_frame.to_csv('../submissions/features.csv')

with open('../submissions/patient_data.pkl', 'wb') as fh:
    pickle.dump(patient_data, fh)

In [6]:
# feature_frame = pd.read_csv('../submissions/features.csv')

# with open('../submissions/patient_data.pkl', 'rb') as fh:
#     patient_data = pickle.load(fh)

## Explore the feature we have extracted

Note: not sure here - trying out something from scikit learn but it may not be sensible.

In [87]:
means = feature_frame.groupby('target').mean()
means

,A,Anti-Coagulant,B,C,Chronic Obstructive Pulmonary Disease (COPD),ChronicIllness,D,Depression,Diabetes,E,...,P,R,T,U,Urology,Z,age,gender,mean_script_time,socio_score
target,,,,,,,,,,,,,,,,,,,,,
0,0.062429,0.011291,0,0.014060,0.031034,0,0,0.178124,0.016513,0,...,0,0,0,0,0.007994,0.002717,0.510163,0.579456,0.055946,0.989366
1,0.068354,0.006958,0,0.018629,0.012792,0,0,0.078509,0.347173,0,...,0,0,0,0,0.005226,0.000377,0.508672,0.493191,0.042682,0.980787


## Try out a set of different classifiers

Note: the feature matrix is usually transformed to have zero mean and unit standard deviation.

In [88]:
X = feature_frame.dropna().drop('target', axis=1).values
y = feature_frame.dropna().target.values

X_transformed = StandardScaler().fit_transform(X)

Partition the data into test and train datasets.

In [89]:
X_train, X_test, y_train, y_test = train_test_split(X_transformed, y, test_size=0.5, random_state=0)

In [90]:
classifiers = [
    ('Random Forrest', RandomForestClassifier()),
    ('Decision Tree', DecisionTreeClassifier()),
    ('Adaboost',AdaBoostClassifier() ),
    ('Gradient boosted decition tree', xgboost.XGBClassifier()), 
    ('SVM',SVC(probability=True))]

for name, clf in classifiers:
    print('Classifier: {}'.format(name))
    model = clf.fit(X_train, y_train)
    y_true, y_pred = y_test, clf.predict(X_test) 
    print(classification_report(y_true, y_pred))


Classifier: Random Forrest
             precision    recall  f1-score   support

          0       0.97      0.96      0.97     40961
          1       0.84      0.84      0.84      8877

avg / total       0.94      0.94      0.94     49838

Classifier: Decision Tree
             precision    recall  f1-score   support

          0       0.95      0.95      0.95     40961
          1       0.78      0.79      0.79      8877

avg / total       0.92      0.92      0.92     49838

Classifier: Adaboost
             precision    recall  f1-score   support

          0       0.98      0.95      0.96     40961
          1       0.78      0.91      0.84      8877

avg / total       0.94      0.94      0.94     49838

Classifier: Gradient boosted decition tree
             precision    recall  f1-score   support

          0       0.98      0.95      0.96     40961
          1       0.80      0.89      0.84      8877

avg / total       0.94      0.94      0.94     49838

Classifier: SVM
       

## Form a submission

Perform the prediction in 1000 patient "chunks" to speed up the processing.

In [97]:
model = xgboost.XGBClassifier().fit(X, y)

In [98]:
submission = pd.read_csv('../submissions/diabetes_submission_example.csv')

In [99]:
chunks = submission.groupby(np.arange(len(submission)) // 1000)

In [100]:
for group, frame in tqdm.tqdm_notebook(chunks):
    
    # Extract the features
    data = [feature_extract(retrieve_patient_data(conn, x)) for x in frame.Patient_ID.values]
    
    # Construct prediction X matrix
    features = pd.DataFrame(data)
    pred_x = features.drop('target', axis=1).values
    pred_x[np.isnan(pred_x)] = 0.0
    
    # Apply the standard transform prior to fitting. 
    pred_x = StandardScaler().fit_transform(pred_x)
    
    # Fit the model
    submission.Diabetes[frame.index] = model.predict_proba(pred_x)[:, 0]

In [96]:
submission.to_csv('../submissions/kaggle_three.csv', index=False)